# 06 computation of confidence and priority order
#    calcul de la confiance et de l'ordre de priorité

In [ ]:
import os, sys, datetime, pandas as pd
sys.path.append("/home/gswinnen/SARSAR_Package_RenPri/code/") # emplacement des modules RenPri

#sys.path.append("/home/issep/sarsar-issep/SARSAR_utils/")                   # emplacement des modules RenPri
#sys.path.append("/home/issep/sarsar-issep/SARSAR_utils/rme_chg_detection_module/") # emplacement de la fonction de Mattia

from issep import sarsar_admin
from os.path import join
from lecture_ini import config

## Function definition: confidence_and_priority_computation

In [ ]:
def confidence_and_priority_computation(dteChgEnd, dteExe, summer, report_type = 'bimestrial'):

    
    # Define Database connection parameters
    # NOTE: password is in ~/.pgpass

    credentials = config(section='postgresql')

    db_credentials = {
        'host': credentials['host'],
        'user': credentials['user'],
        'db' : credentials['database']
    }

    # ALWAYS prepare env et the beginning
    print('> Preparing env (DB credentials, etc)')
    sarsar_admin.prepare_env(db_credentials)
    
    # Ouvre la connexion à la DB
    conn = sarsar_admin._create_or_get_db_connection()
    cur = None

    # Liste de dictionnaires 'sar_id_segment'
    sar_id_segments = []

    try:
        import psycopg2.extras
        cur = conn.cursor(cursor_factory = psycopg2.extras.DictCursor)

        if report_type == "bimestrial" :
                
#             if dteChgEnd >= datetime() :
#                 last_summer = str(summer)+"-"+str(summer-1)
                
    
            print("Report_type = ", report_type, "\nExecution date = ",dteExe,"\nMaximum change date = ", dteChgEnd, "\nLast summer report = ", last_summer)

            ## 1° Création de infos_site
            # Nous permet de lister tous les sites actifs, de prendre leur nom, leur superficie, 
            # la date de changement max (dtechgend)
            cur.execute('DROP VIEW IF EXISTS "infos_sites_{0}" CASCADE;'.format(dteChgEnd))
            conn.commit()

            strSQL = '''CREATE OR REPLACE VIEW "infos_sites_{0}" AS 
                        SELECT id_segment, codecarto, id_sar, shape_area, \'Oui\' AS detection_date, \'Non\' AS detection_ete, \'NA\' AS ete, 
                        \'{0}\' AS date_de_detection FROM sar_def;'''.format(dteChgEnd)
            cur.execute(strSQL)
            conn.commit()

            ## 2° Création d'info_dates_all
            # On fait une jointure entre infos site et les dates de changements détectées, 
            #les amplitudes et classes de changements pour le run d'intérêt défini par dteExe
            cur.execute('DROP VIEW IF EXISTS "info_dates_all_{0}" CASCADE;'.format(dteChgEnd))
            conn.commit()

            strSQL = '''CREATE OR REPLACE VIEW "info_dates_all_{0}" AS 
                        SELECT i.*, (i.date_de_detection::date - \'1 year\'::interval)::date AS "date_detection-1", 
                         (date_de_detection::date - \'2 year\'::interval)::date AS "date_detection-2", 
                          s.date AS date_de_changement, s.vegetation AS changement_vegetation, 
                          s.ndvi_amplitude AS vegetation_amplitude, s.building AS changement_batiment, 
                          s.vh_amplitude AS batiment_amplitude, s.soil AS changement_sol, 
                          s.bai_amplitude AS sol_amplitude, s.nimages_a AS nombre_images_1, 
                          s.nimages_p AS nombre_images_2, 
                          (abs(s.ndvi_amplitude) + abs(s.bai_amplitude) + abs(s.vh_amplitude)) AS amplitude 
                          FROM "infos_sites_{0}" i LEFT JOIN 
                          (SELECT * FROM classif_bimestrial_chg_dates WHERE dteExe = \'{1}\'::date) s 
                          ON s.id_segment = i.id_segment;'''.format(dteChgEnd,f'{dteExe[0:4]}-{dteExe[4:6]}-{dteExe[6:]}')
            print(strSQL)
            cur.execute(strSQL)
            conn.commit()

            ## 3° Création d'info_dates-2
            # On fait une jointure entre infos site et les dates de changements détectées au cours des 2 dernières années 
            # à partir de dteChgEnd
            cur.execute('DROP VIEW IF EXISTS "info_dates_all_{0}_-2" CASCADE;'.format(dteChgEnd))
            conn.commit()

            strSQL = '''CREATE OR REPLACE VIEW "info_dates_all_{0}_-2" AS 
                        SELECT i.*, (i.date_de_detection::date - \'1 year\'::interval)::date AS "date_detection-1", 
                        (i.date_de_detection::date - \'2 year\'::interval)::date AS "date_detection-2", s.date::date AS date_de_changement 
                        FROM "infos_sites_{0}" i LEFT JOIN classif_bimestrial_chg_dates s ON s.id_segment = i.id_segment 
                        WHERE s.date::date BETWEEN (i.date_de_detection::date - \'2 year\'::interval)::date 
                        AND i.date_de_detection::date;'''.format(dteChgEnd)
            #             WHERE date_de_changement::date BETWEEN "date_detection-2"::date AND date_de_detection::date;'''.format(dteChgEnd)

            cur.execute(strSQL)
            conn.commit()

            ## 4° Création de points_dates et tables intermédiaires

            # On calcule le nombre de changement par site entre la dteChgEnd et dteChgEnd -2
            cur.execute('DROP VIEW IF EXISTS "nbr_changements_{0}_-2" CASCADE;'.format(dteChgEnd))
            conn.commit()

            strSQL = '''CREATE OR REPLACE VIEW "nbr_changements_{0}_-2" AS SELECT id_sar, count(*) AS nbr_dates 
                        FROM "info_dates_all_{0}_-2" 
                        WHERE date_de_changement::date BETWEEN "date_detection-2"::date AND date_de_detection::date 
                        GROUP BY id_sar;'''.format(dteChgEnd)
            cur.execute(strSQL)
            conn.commit()

            # On calcule le nombre de non-changement par site entre la dteChgEnd et dteChgEnd -2
            cur.execute('DROP VIEW IF EXISTS "nbr_non-changements_{0}_-2" CASCADE;'.format(dteChgEnd))
            conn.commit()

            strSQL = '''CREATE OR REPLACE VIEW "nbr_non-changements_{0}_-2" AS 
                        SELECT id_sar, 0 AS nbr_dates FROM "infos_sites_{0}" 
                        WHERE id_sar not in (select id_sar FROM "nbr_changements_{0}_-2");'''.format(dteChgEnd)
            cur.execute(strSQL)
            conn.commit()

            ##### TABLE VIDE!

            # On calcule le nombre de changement par site pour la date chg end courante
            cur.execute('DROP VIEW IF EXISTS "nbr_changements_{0}_-1" CASCADE;'.format(dteChgEnd))
            conn.commit()

            strSQL = '''CREATE OR REPLACE VIEW "nbr_changements_{0}_-1" AS 
                        SELECT id_sar, count(*) AS nbr_dates FROM "info_dates_all_{0}" 
                        WHERE date_de_detection::date = \'{0}\'::date 
                        GROUP BY id_sar;'''.format(dteChgEnd)
            cur.execute(strSQL)
            conn.commit()

            # On calcule le nombre de non-changement par site pour la date chg end courante
            cur.execute('DROP VIEW IF EXISTS "nbr_non-changements_{0}_-1" CASCADE;'.format(dteChgEnd))
            conn.commit()

            strSQL = '''CREATE OR REPLACE VIEW "nbr_non-changements_{0}_-1" 
                        AS SELECT ID_SAR, 0 AS nbr_dates FROM "infos_sites_{0}" 
                        WHERE id_sar NOT IN (select ID_SAR FROM "nbr_changements_{0}_-1");'''.format(dteChgEnd)
            cur.execute(strSQL)
            conn.commit()


            # On concatène les vues reprenant les nombres de changements -1 et de non-changements-1
            cur.execute('DROP VIEW IF EXISTS "nbr_dates_{0}_-1" CASCADE;'.format(dteChgEnd))
            conn.commit()

            strSQL = '''CREATE OR REPLACE VIEW "nbr_dates_{0}_-1" AS 
                        SELECT * FROM "nbr_changements_{0}_-1" UNION SELECT * 
                        FROM "nbr_non-changements_{0}_-1";'''.format(dteChgEnd)
            cur.execute(strSQL)
            conn.commit()

            # On concatène les vues reprenant les nombres de changements -2 et de non-changements-2
            cur.execute('DROP VIEW IF EXISTS "nbr_dates_{0}_-2" CASCADE;'.format(dteChgEnd))
            conn.commit()

            strSQL = '''CREATE OR REPLACE VIEW "nbr_dates_{0}_-2" AS 
                        SELECT * FROM "nbr_changements_{0}_-2" union SELECT * 
                        FROM "nbr_non-changements_{0}_-2";'''.format(dteChgEnd)
            cur.execute(strSQL)
            conn.commit()

            # On calcule de nombre de points associés au nombre de dates de changement détectées au cours des deux dernières années à partir de dateChgEnd
            cur.execute('DROP VIEW IF EXISTS "points_dates_{0}" CASCADE;'.format(dteChgEnd))
            conn.commit()

            strSQL = '''CREATE OR REPLACE VIEW "points_dates_{0}" AS SELECT DISTINCT un.id_sar, un.nbr_dates AS "nbr_dates-1", 
                        deux.nbr_dates AS "nbr_dates-2", 
                        (CASE WHEN un.nbr_dates = 0 THEN 3 
                              WHEN un.nbr_dates > 0 AND deux.nbr_dates <= 2 THEN 2 
                              WHEN un.nbr_dates > 0 AND deux.nbr_dates > 2 THEN 1 
                         END) AS points 
                        FROM "nbr_dates_{0}_-1" un, "nbr_dates_{0}_-2" deux WHERE un.id_sar = deux.id_sar;'''.format(dteChgEnd)
            cur.execute(strSQL)
            conn.commit()

            ## 4° Calcul de points_superficie
            cur.execute('DROP VIEW IF EXISTS "points_superficie_{0}" CASCADE;'.format(dteChgEnd))
            conn.commit()

            strSQL = '''CREATE OR REPLACE VIEW "points_superficie_{0}" AS 
                        SELECT DISTINCT id_sar, shape_area, 
                        (CASE WHEN shape_area BETWEEN 500 AND 10000 THEN 1 
                         ELSE 0 
                         END) AS points 
                        FROM "infos_sites_{0}";'''.format(dteChgEnd)
            cur.execute(strSQL)
            conn.commit()

            ## 5° Création de points_changements
            cur.execute('DROP VIEW IF EXISTS "points_changements_{0}" CASCADE;'.format(dteChgEnd))
            conn.commit()

            strSQL = '''CREATE OR REPLACE VIEW "points_changements_{0}" AS 
                        SELECT DISTINCT i.id_sar, i.date_de_changement, i.changement_vegetation, i.changement_batiment, 
                        i.changement_sol, i.nombre_images_1, i.nombre_images_2, n.nbr_dates AS "nbr_dates-1", 
                        (CASE WHEN date_de_changement IS NULL OR n.nbr_dates = 0 THEN 0 
                              WHEN (changement_vegetation + changement_batiment + changement_sol) = 0 THEN 0 
                              WHEN nombre_images_1 > 2 OR nombre_images_2 > 2 THEN 1 
                         ELSE 0
                         END) AS points 
                        FROM "info_dates_all_{0}" i, "nbr_dates_{0}_-1" n WHERE i.ID_SAR = n.ID_SAR;'''.format(dteChgEnd)
            cur.execute(strSQL)
            conn.commit()

            ## 6° Création de infos_dates_all-1
            cur.execute('DROP VIEW IF EXISTS "infos_dates_all_{0}_-1" CASCADE;'.format(dteChgEnd))
            conn.commit()

            strSQL = '''CREATE OR REPLACE VIEW "infos_dates_all_{0}_-1" AS 
                        SELECT DISTINCT i.id_segment, i.codecarto,i.id_sar, i.shape_area,i.detection_date, 
                        i.date_de_detection, i."date_detection-1", i."date_detection-2", 
                        (CASE WHEN d."nbr_dates-1" > 0 THEN i.date_de_changement ELSE NULL END) AS "date_de_changement-1",  
                        (CASE WHEN d."nbr_dates-1" > 0 THEN i.changement_vegetation ELSE NULL END) AS changement_vegetation,
                        (CASE WHEN d."nbr_dates-1" > 0 THEN i.vegetation_amplitude ELSE NULL END) AS vegetation_amplitude, 
                        (CASE WHEN d."nbr_dates-1" > 0 THEN i.changement_sol ELSE NULL END) AS changement_sol, 
                        (CASE WHEN d."nbr_dates-1" > 0 THEN i.sol_amplitude ELSE NULL END) AS sol_amplitude, 
                        (CASE WHEN d."nbr_dates-1" > 0 THEN i.changement_batiment ELSE NULL END) AS changement_batiment, 
                        (CASE WHEN d."nbr_dates-1" > 0 THEN i.batiment_amplitude ELSE NULL END) AS batiment_amplitude, 
                        (CASE WHEN d."nbr_dates-1" > 0 THEN (abs(i.vegetation_amplitude) + abs(i.sol_amplitude) +abs(i.batiment_amplitude)) 
                         ELSE NULL
                         END) AS amplitude, 
                        d."nbr_dates-1", d."nbr_dates-2" 
                        FROM "info_dates_all_{0}" i, "points_dates_{0}" d WHERE d.ID_SAR = i.ID_SAR;'''.format(dteChgEnd)
            cur.execute(strSQL)
            conn.commit()

            ## 7° Création de raw_report_dates_visualisation
            ### DEBUG Il faut encore calculer la "visualisation"
            ### DEBUG Le calcul de visualisation se fait à partir du "raw_report_summer" le plus récent par rapport à la dteChgEnd
            cur.execute('DROP VIEW IF EXISTS "raw_report_dates_visualisation_{0}" CASCADE;'.format(dteChgEnd))
            conn.commit()

            strSQL = '''CREATE OR REPLACE VIEW "raw_report_dates_visualisation_{0}" AS 
                        SELECT DISTINCT i.*, d.points AS points_dates, s.points AS points_superficie, 
                        c.points AS points_changements, (d.points + s.points + c.points) AS confiance, 
                        (CASE WHEN i.amplitude IS NULL THEN 0 
                         ELSE round(((i.amplitude/3.)*((d.points + s.points + c.points)/4.)*100),2)
                         END) AS ordre_de_priorite 
                        FROM "infos_dates_all_{0}_-1" i, "points_dates_{0}" d, "points_superficie_{0}" s, "points_changements_{0}" c 
                        WHERE d.id_sar = i.id_sar AND s.id_sar = i.id_sar AND c.id_sar = i.id_sar order by ordre_de_priorite;'''.format(dteChgEnd)
            cur.execute(strSQL)
            conn.commit()

            ### COWY : je remarque que le calcul de raw_report_dates_selection ne sert à rien. La table n'est réunilisée nulle part ...

            ## 9° Création de bulletin bimestriel
            cur.execute('DROP VIEW IF EXISTS "bimestriel_{0}_DIGIT" CASCADE;'.format(dteChgEnd))
            conn.commit()

            strSQL = '''CREATE OR REPLACE VIEW "bimestriel_{0}_DIGIT" AS SELECT DISTINCT CODECARTO, "nbr_dates-1" AS nbr_dates, 
                        \'NA\' AS detection_estivale, 
                        (CASE WHEN changement_vegetation IS NULL THEN \'Non\'::text 
                              WHEN changement_vegetation = 0 THEN \'Non\'::text
                              WHEN changement_vegetation = 1 OR changement_vegetation = 4 THEN \'Oui\'::text 
                              WHEN changement_vegetation = 2 OR changement_vegetation = 5 THEN \'Oui, augmentation\'::text 
                              WHEN changement_vegetation = 3 OR changement_vegetation = 6 THEN \'Oui, diminution\'::text 
                         ELSE \'Oui\'::text END END END END) AS changement_de_vegetation, 
                        (CASE WHEN changement_batiment IS NULL THEN \'Non\'::text 
                              WHEN changement_batiment = 0 THEN \'Non\'::text
                              WHEN changement_batiment = 1 OR changement_batiment = 4 THEN \'Oui\'::text 
                              WHEN changement_batiment = 2 OR changement_batiment = 5 THEN \'Oui, augmentation\'::text 
                              WHEN changement_batiment = 3 OR changement_batiment = 6 THEN \'Oui, diminution\'::text 
                         ELSE \'Oui\'::text END END END END END) AS changement_de_batiment, 
                        (CASE WHEN changement_sol IS NULL THEN \'Non\'::text 
                              WHEN changement_sol = 0 THEN \'Non\'::text
                              WHEN changement_sol = 1 OR changement_sol = 4 THEN \'Oui\'::text 
                              WHEN changement_sol = 2 OR changement_sol = 5 THEN \'Oui, augmentation\'::text 
                              WHEN changement_sol = 3 OR changement_sol = 6 THEN \'Oui, diminution\'::text 
                         ELSE \'Oui\'::text END END END END END) AS changement_de_sol, 
                        amplitude, confiance, ordre_de_priorite, \'\'::text AS visualisation 
                        FROM "raw_report_dates_visualisation_{0}" ORDER BY ordre_de_priorite DESC;'''.format(dteChgEnd)
            cur.execute(strSQL)
            conn.commit()

            ##ATTENTION, DANS LA DERNIERE VUE, IL Y A PARFOIS PLUSIEURS LIGNES PAR SITES : 
            # ==> il faudra la filtrer à la constitution du rapport ; de sorte de ne garder que la détection avec le plus grand odre de priorité
            # select a.* from "bimestriel_2020-01-01_DIGIT" a inner join (select codecarto, max(ordre_de_priorite) as priorite_max from "bimestriel_2020-01-01_DIGIT" group by codecarto) b on a.codecarto = b.codecarto and a.ordre_de_priorite = b.priorite_max;
                
                
        elif report_type == 'annual' :
            print("Report_type = ", report_type, "\nExecution date = ",dteExe,"\nMaximum change date = ", dteChgEnd, "\nLast summer report = ", last_summer)

            
            ## 1° Création de infos_site
            # Nous permet de lister tous les sites actifs, de prendre leur nom, leur superficie, 
            # la date de changement max (dtechgend)
            cur.execute('DROP VIEW IF EXISTS "infos_sites_{0}" CASCADE;'.format(dteChgEnd))
            conn.commit()

            strSQL = '''CREATE OR REPLACE VIEW "infos_sites_{0}" AS 
                        SELECT id_segment, codecarto, id_sar, shape_area, \'Oui\' AS detection_date, \'Oui\' AS detection_ete, \'\' AS ete, 
                        \'{0}\' AS date_de_detection FROM sar_def;'''.format(dteChgEnd) ### Remplir ete avec le ete de info_summer_all
            cur.execute(strSQL)
            conn.commit()


            ## 2° Création d'info_dates_all
            # On fait une jointure entre infos site et les dates de changements détectées, 
            #les amplitudes et classes de changements pour le run d'intérêt défini par dteExe
            cur.execute('DROP VIEW IF EXISTS "info_dates_all_{0}" CASCADE;'.format(dteChgEnd))
            conn.commit()

            strSQL = '''CREATE OR REPLACE VIEW "info_dates_all_{0}" AS 
                        SELECT i.*, (i.date_de_detection::date - \'1 year\'::interval)::date AS "date_detection-1", 
                         (date_de_detection::date - \'2 year\'::interval)::date AS "date_detection-2", 
                          s.date AS date_de_changement, s.vegetation AS changement_vegetation, 
                          s.ndvi_amplitude AS vegetation_amplitude, s.building AS changement_batiment, 
                          s.vh_amplitude AS batiment_amplitude, s.soil AS changement_sol, 
                          s.bai_amplitude AS sol_amplitude, s.nimages_a AS nombre_images_1, 
                          s.nimages_p AS nombre_images_2, 
                          (abs(s.ndvi_amplitude) + abs(s.bai_amplitude) + abs(s.vh_amplitude)) AS amplitude 
                          FROM "infos_sites_{0}" i LEFT JOIN 
                          (SELECT * FROM classif_bimestrial_chg_dates WHERE dteExe = \'{1}\'::date) s 
                          ON s.id_segment = i.id_segment;'''.format(dteChgEnd,f'{dteExe[0:4]}-{dteExe[4:6]}-{dteExe[6:]}')
            print(strSQL)
            cur.execute(strSQL)
            conn.commit()

            ## 3° Création d'info_dates-2
            # On fait une jointure entre infos site et les dates de changements détectées au cours des 2 dernières années 
            # à partir de dteChgEnd
            cur.execute('DROP VIEW IF EXISTS "info_dates_all_{0}_-2" CASCADE;'.format(dteChgEnd))
            conn.commit()

            strSQL = '''CREATE OR REPLACE VIEW "info_dates_all_{0}_-2" AS 
                        SELECT i.*, (i.date_de_detection::date - \'1 year\'::interval)::date AS "date_detection-1", 
                        (i.date_de_detection::date - \'2 year\'::interval)::date AS "date_detection-2", s.date::date AS date_de_changement 
                        FROM "infos_sites_{0}" i LEFT JOIN classif_bimestrial_chg_dates s ON s.id_segment = i.id_segment 
                        WHERE s.date::date BETWEEN (i.date_de_detection::date - \'2 year\'::interval)::date 
                        AND i.date_de_detection::date;'''.format(dteChgEnd)
            #             WHERE date_de_changement::date BETWEEN "date_detection-2"::date AND date_de_detection::date;'''.format(dteChgEnd)

            cur.execute(strSQL)
            conn.commit()

            ## 4° Création de points_dates et tables intermédiaires

            # On calcule le nombre de changement par site entre la dteChgEnd et dteChgEnd -2
            cur.execute('DROP VIEW IF EXISTS "nbr_changements_{0}_-2" CASCADE;'.format(dteChgEnd))
            conn.commit()

            strSQL = '''CREATE OR REPLACE VIEW "nbr_changements_{0}_-2" AS SELECT id_sar, count(*) AS nbr_dates 
                        FROM "info_dates_all_{0}_-2" 
                        WHERE date_de_changement::date BETWEEN "date_detection-2"::date AND date_de_detection::date 
                        GROUP BY id_sar;'''.format(dteChgEnd)
            cur.execute(strSQL)
            conn.commit()

            # On calcule le nombre de non-changement par site entre la dteChgEnd et dteChgEnd -2
            cur.execute('DROP VIEW IF EXISTS "nbr_non-changements_{0}_-2" CASCADE;'.format(dteChgEnd))
            conn.commit()

            strSQL = '''CREATE OR REPLACE VIEW "nbr_non-changements_{0}_-2" AS 
                        SELECT id_sar, 0 AS nbr_dates FROM "infos_sites_{0}" 
                        WHERE id_sar not in (select id_sar FROM "nbr_changements_{0}_-2");'''.format(dteChgEnd)
            cur.execute(strSQL)
            conn.commit()


            # On calcule le nombre de changement par site pour la date chg end courante
            cur.execute('DROP VIEW IF EXISTS "nbr_changements_{0}_-1" CASCADE;'.format(dteChgEnd))
            conn.commit()

            strSQL = '''CREATE OR REPLACE VIEW "nbr_changements_{0}_-1" AS 
                        SELECT id_sar, count(*) AS nbr_dates FROM "info_dates_all_{0}" 
                        WHERE date_de_detection::date = \'{0}\'::date 
                        GROUP BY id_sar;'''.format(dteChgEnd)
            cur.execute(strSQL)
            conn.commit()

            # On calcule le nombre de non-changement par site pour la date chg end courante
            cur.execute('DROP VIEW IF EXISTS "nbr_non-changements_{0}_-1" CASCADE;'.format(dteChgEnd))
            conn.commit()

            strSQL = '''CREATE OR REPLACE VIEW "nbr_non-changements_{0}_-1" 
                        AS SELECT ID_SAR, 0 AS nbr_dates FROM "infos_sites_{0}" 
                        WHERE id_sar NOT IN (select ID_SAR FROM "nbr_changements_{0}_-1");'''.format(dteChgEnd)
            cur.execute(strSQL)
            conn.commit()


            # On concatène les vues reprenant les nombres de changements -1 et de non-changements-1
            cur.execute('DROP VIEW IF EXISTS "nbr_dates_{0}_-1" CASCADE;'.format(dteChgEnd))
            conn.commit()

            strSQL = '''CREATE OR REPLACE VIEW "nbr_dates_{0}_-1" AS 
                        SELECT * FROM "nbr_changements_{0}_-1" UNION SELECT * 
                        FROM "nbr_non-changements_{0}_-1";'''.format(dteChgEnd)
            cur.execute(strSQL)
            conn.commit()

            # On concatène les vues reprenant les nombres de changements -2 et de non-changements-2
            cur.execute('DROP VIEW IF EXISTS "nbr_dates_{0}_-2" CASCADE;'.format(dteChgEnd))
            conn.commit()

            strSQL = '''CREATE OR REPLACE VIEW "nbr_dates_{0}_-2" AS 
                        SELECT * FROM "nbr_changements_{0}_-2" union SELECT * 
                        FROM "nbr_non-changements_{0}_-2";'''.format(dteChgEnd)
            cur.execute(strSQL)
            conn.commit()

            # On calcule de nombre de points associés au nombre de dates de changement détectées au cours des deux dernières années à partir de dateChgEnd
            cur.execute('DROP VIEW IF EXISTS "points_dates_{0}" CASCADE;'.format(dteChgEnd))
            conn.commit()

            strSQL = '''CREATE OR REPLACE VIEW "points_dates_{0}" AS SELECT DISTINCT un.id_sar, un.nbr_dates AS "nbr_dates-1", 
                        deux.nbr_dates AS "nbr_dates-2", 
                        (CASE WHEN un.nbr_dates = 0 THEN 0 
                              WHEN un.nbr_dates > 0 AND deux.nbr_dates <= 2 THEN 2 
                              WHEN un.nbr_dates > 0 AND deux.nbr_dates > 2 THEN 1
                         END) AS points 
                        FROM "nbr_dates_{0}_-1" un, "nbr_dates_{0}_-2" deux WHERE un.id_sar = deux.id_sar;'''.format(dteChgEnd)
            cur.execute(strSQL)
            conn.commit()

    ##NB : les points_dates = 0 (nbr_dates = 0) sont comparés plus tard à points_summer dans la variable points_nochange

            ## 4° Calcul de points_superficie
            cur.execute('DROP VIEW IF EXISTS "points_superficie_{0}" CASCADE;'.format(dteChgEnd))
            conn.commit()

            strSQL = '''CREATE OR REPLACE VIEW "points_superficie_{0}" AS 
                        SELECT DISTINCT id_sar, shape_area, 
                        (CASE WHEN shape_area BETWEEN 500 AND 10000 THEN 1 
                         ELSE 0
                         END) AS points 
                        FROM "infos_sites_{0}";'''.format(dteChgEnd)
            cur.execute(strSQL)
            conn.commit()

            ## 5° Création de points_changements
            cur.execute('DROP VIEW IF EXISTS "points_changements_{0}" CASCADE;'.format(dteChgEnd))
            conn.commit()

            strSQL = '''CREATE OR REPLACE VIEW "points_changements_{0}" AS 
                        SELECT DISTINCT i.id_sar, i.date_de_changement, i.changement_vegetation, i.changement_batiment, 
                        i.changement_sol, i.nombre_images_1, i.nombre_images_2, n.nbr_dates AS "nbr_dates-1", 
                        (CASE WHEN date_de_changement IS NULL OR n.nbr_dates = 0 THEN 0 
                              WHEN (changement_vegetation + changement_batiment + changement_sol) = 0 THEN 0 
                              WHEN nombre_images_1 > 2 OR nombre_images_2 > 2 THEN 1 
                         ELSE 0
                         END) AS points 
                        FROM "info_dates_all_{0}" i, "nbr_dates_{0}_-1" n WHERE i.ID_SAR = n.ID_SAR;'''.format(dteChgEnd)
            cur.execute(strSQL)
            conn.commit()

            ## 6° Création de infos_dates_all-1
            cur.execute('DROP VIEW IF EXISTS "infos_dates_all_{0}_-1" CASCADE;'.format(dteChgEnd))
            conn.commit()

            strSQL = '''CREATE OR REPLACE VIEW "infos_dates_all_{0}_-1" AS 
                        SELECT DISTINCT i.id_segment, i.codecarto,i.id_sar, i.shape_area,i.detection_date, 
                        i.date_de_detection, i."date_detection-1", i."date_detection-2", 
                        (CASE WHEN d."nbr_dates-1" > 0 THEN i.date_de_changement ELSE NULL END) AS "date_de_changement-1",  
                        (CASE WHEN d."nbr_dates-1" > 0 THEN i.changement_vegetation ELSE NULL END) AS changement_vegetation,
                        (CASE WHEN d."nbr_dates-1" > 0 THEN i.vegetation_amplitude ELSE NULL END) AS vegetation_amplitude, 
                        (CASE WHEN d."nbr_dates-1" > 0 THEN i.changement_sol ELSE NULL END) AS changement_sol, 
                        (CASE WHEN d."nbr_dates-1" > 0 THEN i.sol_amplitude ELSE NULL END) AS sol_amplitude, 
                        (CASE WHEN d."nbr_dates-1" > 0 THEN i.changement_batiment ELSE NULL END) AS changement_batiment, 
                        (CASE WHEN d."nbr_dates-1" > 0 THEN i.batiment_amplitude ELSE NULL END) AS batiment_amplitude, 
                        (CASE WHEN d."nbr_dates-1" > 0 THEN (abs(i.vegetation_amplitude) + abs(i.sol_amplitude) +abs(i.batiment_amplitude)) 
                         ELSE NULL END) AS amplitude, 
                        d."nbr_dates-1", d."nbr_dates-2" 
                        FROM "info_dates_all_{0}" i, "points_dates_{0}" d WHERE d.ID_SAR = i.ID_SAR;'''.format(dteChgEnd)
            cur.execute(strSQL)
            conn.commit()

            ## 7° Création de raw_report_dates
            cur.execute('DROP VIEW IF EXISTS "raw_report_dates_visualisation_{0}" CASCADE;'.format(dteChgEnd))
            conn.commit()

            strSQL = '''CREATE OR REPLACE VIEW "raw_report_dates_visualisation_{0}" AS 
                        SELECT DISTINCT i.*, d.points AS points_dates, s.points AS points_superficie, 
                        c.points AS points_changements, (d.points + s.points + c.points) AS confiance, 
                        (CASE WHEN i.amplitude IS NULL THEN 0 
                         ELSE round(((i.amplitude/3.)*((d.points + s.points + c.points)/4.)*100),2) END) AS ordre_de_priorite 
                        FROM "infos_dates_all_{0}_-1" i, "points_dates_{0}" d, "points_superficie_{0}" s, "points_changements_{0}" c 
                        WHERE d.id_sar = i.id_sar AND s.id_sar = i.id_sar AND c.id_sar = i.id_sar order by ordre_de_priorite;'''.format(dteChgEnd)
            cur.execute(strSQL)
            conn.commit()

            ## 8° Création de info_summer_all
            ### On va chercher dans la table "classification_chg_summer" les lignes pour lesquelles "periode" = "summer-(summer-1)"
            # On fait une jointure avec infos site, 
            #les amplitudes et classes de changements pour le run d'intérêt défini par dteExe
            cur.execute('DROP VIEW IF EXISTS "info_summer_all_{0}" CASCADE;'.format(dteChgEnd))
            conn.commit()

            strSQL = '''CREATE OR REPLACE VIEW "info_summer_all_{0}" AS 
                        SELECT a.id_segment, a.periode AS ete,
                          a.vegetation AS changement_vegetation, a.ndvi_amplitude AS vegetation_amplitude, 
                          a.building AS changement_batiment, a.compo_amplitude AS batiment_amplitude,
                          a.soil AS changement_sol, a.bai_amplitude AS sol_amplitude, 
                          (abs(a.ndvi_amplitude) + abs(a.bai_amplitude) + abs(a.compo_amplitude)) AS amplitude 
                          FROM 
                          (SELECT * FROM classification_chg_summer WHERE dteExe = \'{1}\'::date) a ;'''.format(dteChgEnd,f'{dteExe[0:4]}-{dteExe[4:6]}-{dteExe[6:]}')
            print(strSQL)
            cur.execute(strSQL)
            conn.commit()

            ## 9° Création de points_summer
            ### à partir de infos_summer_all
            ### Si vegetation +soil + building =0 alors 0 pt
            ### else Si vegetation, soil ou building comprenne au moins un changement tout court alors 2pts
            ### else 1pt (vegetation, soil ou building comprenne ne comprenne que des changements probable alors 1pt)
            cur.execute('DROP VIEW IF EXISTS "points_summer_{0}" CASCADE;'.format(dteChgEnd))
            conn.commit()

            strSQL = '''CREATE OR REPLACE VIEW "points_summer_{0}" AS 
                        SELECT a.id_segment,
                          (CASE WHEN a.changement_vegetation = 0 AND a.changement_batiment = 0 AND a.changement_sol = 0 THEN 0
                                WHEN a.changement_vegetation = 2 OR a.changement_vegetation = 3 OR a.changement_batiment = 1 OR a.changement_sol = 1 THEN 2
                                WHEN a.changement_vegetation = 5 OR a.changement_vegetation = 6 OR a.changement_batiment = 4 OR a.changement_sol = 4 THEN 1
                           ELSE NULL
                           END) AS points_summer 
                          FROM "info_summer_all_{0}" a ;'''.format(dteChgEnd)
            print(strSQL)
            cur.execute(strSQL)
            conn.commit()

            ## 10° Création de points_nochange
            ### Si pour un site, points_dates(raw_report_dates) et points_summer(points_summer) = 0 alors 5 pts
            cur.execute('DROP VIEW IF EXISTS "points_nochange_{0}" CASCADE;'.format(dteChgEnd))
            conn.commit()

            strSQL = '''CREATE OR REPLACE VIEW "points_nochange_{0}" AS 
                        SELECT b.id_segment,
                          (CASE WHEN a.points_dates = 0 AND b.points_summer = 0 THEN 5
                           ELSE 0
                           END) AS points_nochange
                          FROM "raw_report_dates_{0}" a, "points_summer_{0}" b 
                          WHERE a.id_segment = b.id_segment;'''.format(dteChgEnd)
            print(strSQL)
            cur.execute(strSQL)
            conn.commit()
            ## 11° Création de raw_report_summer
            ### fusion de infos summer_all, points_summer, points_no_change + calcul confiance + calcul ordre de priorité
            ### reprends points_superficie points_dates points_changements de raw report dates

            cur.execute('DROP VIEW IF EXISTS "raw_report_summer_{0}" CASCADE;'.format(dteChgEnd))
            conn.commit()

            strSQL = '''CREATE OR REPLACE VIEW "raw_report_summer_{0}" AS 
                        SELECT a.*,
                          b.points_summer, c.points_nochange, d.points_superficie, d.points_dates
                          (b.points_summer + c.points_nochange + d.points_superficie + d.points_dates + d.points_changements) AS confiance,
                          (CASE WHEN a.amplitude IS NULL THEN 0 
                           ELSE round(((a.amplitude/3.)*((b.points_summer + c.points_nochange + d.points_superficie + d.points_dates + d.points_changements)/6.)*100),2)
                           END) AS ordre_de_priorite 
                          FROM "info_summer_all_{0}" a, "points_summer_{0}" b, "points_nochange_{0}" c, "raw_report_dates_{0}" d
                          WHERE a.id_segment = b.id_segment AND a.id_segment = c.id_segment AND a.id_segment = d.id_segment;'''.format(dteChgEnd)
            print(strSQL)
            cur.execute(strSQL)
            conn.commit()

            ## 12° Creation de infos_year_all : synthèse de raw_report_dates et de raw_report_summer
            cur.execute('DROP VIEW IF EXISTS "info_year_all_{0}" CASCADE;'.format(dteChgEnd))
            conn.commit()

            strSQL = '''CREATE OR REPLACE VIEW "info_year_all_{0}" AS 
                        SELECT a.codecarto, a.shape_area, a.detection_date, a.date_de_detection, a."nbr_dates-1", a."date_de_changement-1",
                        a.changement_vegetation AS changement_vegetation_d, a.changement_batiment AS changement_batiment_d,a.changement_sol AS changement_sol_d,
                        a.amplitude AS amplitude_d,a.confiance AS confiance_d, a.ordre_de_priorite_d,
                        b.ete
                        b.changement_vegetation AS changement_vegetation_e, b.changement_batiment AS changement_batiment_e,b.changement_sol AS changement_sol_e,
                        b.amplitude AS amplitude_e,b.confiance AS confiance_e, b.ordre_de_priorite_e,

                        FROM raw_report_dates_{0}" a, "raw_report_summer_{0}" b
                        WHERE a.id_segment = b.id_segment ORDRE BY a.codecarto ASC;'''.format(dteChgEnd)
            print(strSQL)
            cur.execute(strSQL)
            conn.commit()
            
            ## 13° Création de raw_report_year
            ### les colonnes "*dates" et '*summer' sont comparées pour prendre les plus grands changements, amplitude, confiance, ordre de priorité
            ### le champ visualisation est calculé
            cur.execute('DROP VIEW IF EXISTS "raw_report_year_{0}" CASCADE;'.format(dteChgEnd))
            conn.commit()

            strSQL = '''CREATE OR REPLACE VIEW "raw_report_year_{0}" AS 
                        SELECT a.CODECARTO, a."nbr_dates-1" as nbr_dates, \'Oui\' AS detection_estivale,
                        (CASE WHEN a.changement_vegetation = 0 AND b.changement_vegetation = 0 THEN 0
                              WHEN a.changement_vegetation = 0 AND b.changement_vegetation != 0 THEN b.changement_vegetation
                              WHEN a.changement_vegetation != 0 AND b.changement_vegetation = 0 THEN a.changement_vegetation
                              WHEN a.changement_vegetation = 2 AND (b.changement_vegetation = 2 OR b.changement_vegetation = 5) THEN a.changement
                              WHEN a.changement_vegetation = 2 AND (b.changement_vegetation = 3 OR b.changement_vegetation = 6) THEN 1
                              WHEN a.changement_vegetation = 3 AND (b.changement_vegetation = 2 OR b.changement_vegetation = 5) THEN 1
                              WHEN a.changement_vegetation = 3 AND (b.changement_vegetation = 3 OR b.changement_vegetation = 6) THEN a.changement
                         END) AS changement_de_vegetation
                        (CASE WHEN changement_de_sol = 0 AND b.changement_de_sol = 0 THEN 0
                              WHEN a.changement_de_sol = 0 AND b.changement_de_sol != 0 THEN b.changement_de_sol
                              WHEN a.changement_de_sol != 0 AND b.changement_de_sol = 0 THEN a.changement_de_sol
                              WHEN a.changement_de_sol = 1 AND (b.changement_de_sol = 1 OR b.changement_de_sol = 4) THEN a.changement_de_sol
                         END) AS changement_de_batiment
                        (CASE WHEN changement_de_batiment = 0 AND b.changement_de_batiment = 0 THEN 0
                              WHEN a.changement_de_batiment = 0 AND b.changement_de_batiment != 0 THEN b.changement_de_batiment
                              WHEN a.changement_de_batiment != 0 AND b.changement_de_batiment = 0 THEN a.changement_de_batiment
                              WHEN a.changement_de_batiment = 2 AND (b.changement_de_batiment = 1 OR b.changement_de_batiment = 4) THEN a.changement_de_batiment
                              WHEN a.changement_de_batiment = 3 AND (b.changement_de_batiment = 1 OR b.changement_de_batiment = 4) THEN a.changement_de_batiment
                         END) AS changement_de_batiment
                        (CASE WHEN a.amplitude > b.amplitude THEN a.amplitude
                              ELSE b.amplitude
                         END) AS amplitude,
                        (CASE WHEN a.confiance > b.confiance THEN a.confiance
                              ELSE b.confiance
                         END) AS confiance,
                        (CASE WHEN a.ordre_de_priorite > b.ordre_de_priorite THEN a.ordre_de_priorite
                              ELSE b.ordre_de_priorite
                        END) AS ordre_de_priorite,
                        (CASE WHEN a."nbr_dates-1" = 0 AND b.points_summer = 0 THEN "Pas de changement"
                              WHEN a."nbr_dates-1" > 0 AND b.points_summer > 0 THEN "Changement"
                              WHEN a."nbr_dates-1" > 0 OR b.points_summer > 0 THEN "Changement possible"
                         END) AS visualisation
                        FROM "raw_report_dates_{0}" a, FROM "raw_report_summer_{0}" b
                        WHERE a.id_segment = b.id_segment;'''.format(dteChgEnd)
            print(strSQL)
            cur.execute(strSQL)
            conn.commit()

            ## 9° Création de bulletin annuel avec le champ visualisation
            cur.execute('DROP VIEW IF EXISTS annuel_{0}_DIGIT" CASCADE;'.format(dteChgEnd))
            conn.commit()

            strSQL = '''CREATE OR REPLACE VIEW "annuel_{0}_DIGIT" AS SELECT DISTINCT a.CODECARTO,
                        a.nbr_dates, a.detection_estivale, 
                        (CASE WHEN a.changement_de_vegetation IS NULL THEN \'Non\'::text 
                              WHEN a.changement_de_vegetation = 0 THEN \'Non\'::text
                              WHEN a.changement_de_vegetation= 1 OR a.changement_de_vegetation= 4 THEN \'Oui\'::text 
                              WHEN a.changement_de_vegetation = 2 OR a.changement_de_vegetation = 5 THEN \'Oui, augmentation\'::text 
                              WHEN a.changement_de_vegetation= 3 OR a.changement_de_vegetation = 6 THEN \'Oui, diminution\'::text
                         END) AS changement_de_vegetation, 
                        (CASE WHEN a.changement_de_batiment IS NULL THEN \'Non\'::text 
                              WHEN a.changement_de_batiment = 0 THEN \'Non\'::text
                              WHEN a.changement_de_batiment = 1 OR a.changement_de_batiment = 4 THEN \'Oui\'::text 
                              WHEN a.changement_de_batiment= 2 OR a.changement_de_batiment= 5 THEN \'Oui, augmentation\'::text 
                              WHEN a.changement_de_batiment= 3 OR a.changement_de_batiment = 6 THEN \'Oui, diminution\'::text 
                         END) AS changement_de_batiment, 
                        (CASE WHEN a.changement_sol IS NULL THEN \'Non\'::text 
                              WHEN a.changement_sol = 0 THEN \'Non\'::text
                              WHEN a.changement_sol = 1 OR a.changement_sol = 4 THEN \'Oui\'::text 
                              WHEN a.changement_sol = 2 OR a.changement_sol = 5 THEN \'Oui, augmentation\'::text 
                              WHEN a.changement_sol = 3 OR a.changement_sol = 6 THEN \'Oui, diminution\'::text 
                         END) AS changement_de_sol, 
                        a.amplitude, a.confiance, a.ordre_de_priorite, a.visualisation 
                        FROM "raw_report_year_{0}" a ORDER BY ordre_de_priorite DESC;'''.format(dteChgEnd)
            cur.execute(strSQL)
            conn.commit()
            
            
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

    finally:
        if cur is not None:
            cur.close()
            
    # ALWAYS release env at the end
    print('> Releasing env')
    sarsar_admin.release_env()

## Bulletin annuel